# Import

In [1]:
import os
import nltk
import json
from time import gmtime, strftime
from nltk import sent_tokenize
from tqdm import tqdm
from itertools import chain

import multiprocessing
import parmap


from datasets import load_dataset, load_from_disk, concatenate_datasets
from transformers import AutoTokenizer, AutoConfig, TrainingArguments, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer
from accelerate import notebook_launcher

import pickle

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# Load Dataset

In [2]:
data_path = "./data"
model_cache_dir = "./BERT_cache"
current_time = strftime("%Y-%m-%d-%H:%M:%S", gmtime())
model_output_dir = './BERT'+"_"+current_time
original_train_datasets_path = data_path + "/original_datasets/TS1"
original_valid_datasets_path = data_path + "/original_datasets/VS1"
train_sentence_list_file_name = "train_sentence_list.txt"
valid_sentence_list_file_name = "valid_sentence_list.txt"
# raw_datasets_folder_name = "raw_datasets"
raw_datasets_cache_name = ".raw_datasets_cache"
# raw_datasets_path = os.path.join(data_path, raw_datasets_folder_name)
raw_datasets_cache_path = os.path.join(data_path, raw_datasets_cache_name)
tokenizer_name = "tokenizer_aihub_news"
tokenized_datasets_folder_name = ["tokenized_datasets", "tokenized_datasets_1", "tokenized_datasets_2", "tokenized_datasets_3"]
grouped_tokenized_datasets_folder_name = "grouped_tokenized_datasets"

In [3]:
num_proc = multiprocessing.cpu_count()

In [4]:
# def listize_dataset(json_file_path):    
#     sentence_list = []

#     with open(json_file_path, 'r', encoding='UTF-8') as f:
#         json_object = json.load(f)

#     for line in json_object['SJML']['text']:
#         raw_text = line['content'].replace('..', '.')
#         raw_text_list = sent_tokenize(raw_text)
        
#         for item in raw_text_list:
#             if len(item) < 5:
#                 continue
#             else:
#                 sentence_list.append(item.strip())

#     return sentence_list

In [5]:
# file_path_list = []

# file_list = os.listdir(os.path.join(original_train_datasets_path))
# for dir_name in file_list:
#     file_path = os.path.join(original_train_datasets_path, dir_name)

#     json_file_list = os.listdir(file_path)
#     for json_file_name in json_file_list:
#         json_file_path = os.path.join(file_path, json_file_name)

#         file_path_list.append(json_file_path)

# train_sentence_list_of_list = parmap.map(listize_dataset, file_path_list, pm_pbar=True, pm_processes=num_proc)

In [6]:
# file_path_list = []
# file_list = os.listdir(os.path.join(original_valid_datasets_path))
# for dir_name in file_list:
#     file_path = os.path.join(original_valid_datasets_path, dir_name)

#     json_file_list = os.listdir(file_path)
#     for json_file_name in json_file_list:
#         json_file_path = os.path.join(file_path, json_file_name)

#         file_path_list.append(json_file_path)

# valid_sentence_list_of_list = parmap.map(listize_dataset, file_path_list, pm_pbar=True, pm_processes=num_proc)

In [7]:
# train_sentence_list = list(chain(*train_sentence_list_of_list))
# valid_sentence_list = list(chain(*valid_sentence_list_of_list))

In [8]:
# with open(os.path.join(data_path, train_sentence_list_file_name), 'w') as fw:
#     for sentence in train_sentence_list:
#         fw.write(sentence + "\n")

# with open(os.path.join(data_path, valid_sentence_list_file_name), 'w') as fw:
#     for sentence in valid_sentence_list:
#         fw.write(sentence + "\n")

In [9]:
raw_datasets = load_dataset('text', data_files={"train": os.path.join(data_path, train_sentence_list_file_name), "valid": os.path.join(data_path, valid_sentence_list_file_name)}, cache_dir=raw_datasets_cache_path)

Found cached dataset text (/home/work/sekim_backup/BERT_pretrain/data/.raw_datasets_cache/text/default-a8c285c79804c5c2/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# raw_datasets

In [11]:
# print(len(raw_datasets))
# raw_datasets['train'][10]

# Tokenizing

In [12]:
# # training a tokenizer from scratch
# def batch_iterator(batch_size=10000):
#     for i in tqdm(range(0, len(raw_datasets), batch_size)):
#         yield raw_datasets[i:i+batch_size]["text"]
        
# old_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = old_tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=15000)
# tokenizer.save_pretrained(tokenizer_name)

In [13]:
# raw_datasets_1 = raw_datasets.select(range(20000000))
# raw_datasets_2 = raw_datasets.select(range(20000000, 40000000))
# raw_datasets_3 = raw_datasets.select(range(40000000, len(raw_datasets)))

In [14]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [15]:
# def preprocess_texts(examples):
#     tokenized_inputs = tokenizer(
#        examples["text"], return_special_tokens_mask=True, truncation=True, max_length=tokenizer.model_max_length
#     )
#     return tokenized_inputs

In [16]:
# tokenized_datasets = raw_datasets.map(preprocess_texts, batched=True, remove_columns=["text"], num_proc=num_proc)

In [17]:
# tokenized_datasets_1 = raw_datasets_1.map(preprocess_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
# tokenized_datasets_1.save_to_disk(os.path.join(data_path, tokenized_datasets_folder_name[1]))

In [18]:
# tokenized_datasets_2 = raw_datasets_2.map(preprocess_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
# tokenized_datasets_2.save_to_disk(os.path.join(data_path, tokenized_datasets_folder_name[2]))

In [19]:
# tokenized_datasets_3 = raw_datasets_3.map(preprocess_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
# tokenized_datasets_3.save_to_disk(os.path.join(data_path, tokenized_datasets_folder_name[3]))

In [20]:
# tokenized_datasets_1 = load_from_disk(os.path.join(data_path, tokenized_datasets_folder_name[1]))

In [21]:
# tokenized_datasets_2 = load_from_disk(os.path.join(data_path, tokenized_datasets_folder_name[2]))

In [22]:
# tokenized_datasets_3 = load_from_disk(os.path.join(data_path, tokenized_datasets_folder_name[3]))

In [23]:
# tokenized_datasets = concatenate_datasets([tokenized_datasets_1, tokenized_datasets_2, tokenized_datasets_1])

In [24]:
# tokenized_datasets

In [25]:
# tokenized_datasets.save_to_disk(os.path.join(data_path, tokenized_datasets_folder_name[0]))

In [26]:
# tokenized_datasets = load_from_disk(os.path.join(data_path, tokenized_datasets_folder_name[0]))

# Grouping

In [27]:
# # Main data processing function that will concatenate all texts from our dataset and generate chunks of
# # max_seq_length.
# def group_texts(examples):
#     # Concatenate all texts.
#     concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
#     total_length = len(concatenated_examples[list(examples.keys())[0]])
#     # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
#     # customize this part to your needs.
#     if total_length >= tokenizer.model_max_length:
#         total_length = (total_length // tokenizer.model_max_length) * tokenizer.model_max_length
#     # Split by chunks of max_len.
#     result = {
#         k: [t[i : i + tokenizer.model_max_length] for i in range(0, total_length, tokenizer.model_max_length)]
#         for k, t in concatenated_examples.items()
#     }
#     return result

In [28]:
# tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=16)
# # shuffle dataset
# tokenized_datasets = tokenized_datasets.shuffle(seed=34)

# print(f"the dataset contains in total {len(tokenized_datasets)*tokenizer.model_max_length} tokens")

In [29]:
# tokenized_datasets

In [30]:
# tokenized_datasets.save_to_disk(os.path.join(data_path, grouped_tokenized_datasets_folder_name))

In [31]:
tokenized_datasets = load_from_disk(os.path.join(data_path, grouped_tokenized_datasets_folder_name))

# DDP Train

In [32]:
model_config = AutoConfig.from_pretrained('bert-base-uncased')

In [33]:
# DDP Train
def train_trainer_ddp():
    model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased', config=model_config, cache_dir=model_cache_dir)
    model.resize_token_embeddings(len(tokenizer))

    training_args = TrainingArguments(
        output_dir = model_output_dir,
        logging_dir="runs/"+model_output_dir,
        do_train = True,
        do_eval = True,
        per_device_train_batch_size = 48,
        per_device_eval_batch_size = 48,        
        evaluation_strategy = "steps",
        eval_steps=1000,
        save_strategy="steps",
        save_steps=5000,
        logging_steps = 100,
        learning_rate = 5e-5,
        weight_decay = 0,
        adam_epsilon = 1e-8,
        max_grad_norm = 1.0,
        num_train_epochs = 2,
        disable_tqdm="false",
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm=True, 
                                                mlm_probability=0.15,)

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['valid'][:100000]
    )   

    trainer.train()

notebook_launcher(train_trainer_ddp, args=(), num_processes=4)

Launching training on 4 GPUs.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model

{'loss': 8.0203, 'learning_rate': 4.988613072193122e-05, 'epoch': 0.0}
{'loss': 6.324, 'learning_rate': 4.9772261443862454e-05, 'epoch': 0.01}
{'loss': 5.8192, 'learning_rate': 4.965839216579367e-05, 'epoch': 0.01}
{'loss': 5.4793, 'learning_rate': 4.954452288772489e-05, 'epoch': 0.02}
{'loss': 5.1615, 'learning_rate': 4.943065360965612e-05, 'epoch': 0.02}
{'loss': 4.8564, 'learning_rate': 4.931678433158734e-05, 'epoch': 0.03}
{'loss': 4.6338, 'learning_rate': 4.920291505351856e-05, 'epoch': 0.03}
{'loss': 4.459, 'learning_rate': 4.9089045775449787e-05, 'epoch': 0.04}
{'loss': 4.3045, 'learning_rate': 4.897517649738101e-05, 'epoch': 0.04}
{'loss': 4.1715, 'learning_rate': 4.886130721931223e-05, 'epoch': 0.05}
{'eval_input_ids_loss': 3.972381591796875, 'eval_input_ids_runtime': 75.2814, 'eval_input_ids_samples_per_second': 1328.35, 'eval_input_ids_steps_per_second': 6.921, 'epoch': 0.05}
{'eval_token_type_ids_loss': nan, 'eval_token_type_ids_runtime': 74.717, 'eval_token_type_ids_sample

# Prepare Training

In [34]:
# model_config = AutoConfig.from_pretrained('bert-base-uncased')

In [35]:
# model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased', config=model_config, cache_dir=model_cache_dir)
# model.resize_token_embeddings(len(tokenizer))

In [36]:
# training_args = TrainingArguments(
#     output_dir = model_output_dir,
#     overwrite_output_dir = True,
#     do_train = True,
#     do_eval = True,
#     per_device_train_batch_size = 32,
#     per_device_eval_batch_size = 32,
#     logging_steps = 50,
#     prediction_loss_only = True,
#     learning_rate = 5e-5,
#     weight_decay = 0,
#     adam_epsilon = 1e-8,
#     max_grad_norm = 1.0,
#     num_train_epochs = 2,
#     save_steps = -1
# )

In [37]:
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
#                                                 mlm=True, 
#                                                 mlm_probability=0.15,)

In [38]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=tokenized_datasets,
# )

In [39]:
# trainer.train()